In [1]:
import pandas as pd
import numpy as np
import time
import gc

In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
dtypes = {
        'ip'            : 'uint64',
        'app'           : 'uint64',
        'device'        : 'uint64',
        'os'            : 'uint64',
        'channel'       : 'uint64',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

train = pd.read_csv(path + 'train.csv',dtype=dtypes)
print('train done!')
# test = pd.read_csv(path + 'test.csv',dtype=dtypes)
# print('test done!')
# test_supplement = pd.read_csv(path + 'test_supplement.csv',dtype=dtypes)
# print('supplement done!')


train done!


In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
temp = pd.read_csv(path+"supplement2testInexFinal.csv")

In [3]:
temp.head(20)

,test_click,supplement_click
0,0,21290878
1,1,21290876
2,2,21290877
3,3,21290879
4,4,21290880
5,5,21290882
6,6,21290881
7,7,21290883
8,9,21290884
9,8,21290885


In [6]:
temp.to_feather(fname='/home/kai/data/kaggle/talkingdata/data/feature_tmp.ftr')

In [7]:
rr = pd.read_feather('/home/kai/data/kaggle/talkingdata/data/feature_tmp.ftr',dr)

In [8]:
rr

,test_click,supplement_click
0,0,21290878
1,1,21290876
2,2,21290877
3,3,21290879
4,4,21290880
5,5,21290882
6,6,21290881
7,7,21290883
8,9,21290884
9,8,21290885


# Process Timestamp

In [155]:
import multiprocessing
workers = 20

def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop('workers')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))

def get_timestamp(x):
    return x.timestamp()

for df in [test_supplement,test]:
    clicks = pd.to_datetime(df.click_time)
    print('get clicks')
    df['timestamp'] = apply_by_multiprocessing(clicks, get_timestamp, workers=workers)
    #     df['timestamp'] = clicks.apply(lambda t: t.timestamp())
    df['timestamp'] = df['timestamp'].astype('uint64')
    print('timestamping is done')

get clicks
timestamping is done
get clicks
timestamping is done


In [172]:
orders = {}
feature_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'timestamp']

# feature_col = ['ip', 
#               'app', 
#               'device', 
#               'os', 
#               'channel']
for col in feature_col:
    orders[col] = 10 ** (int(np.log(max(test_supplement[col].max(),test[col].max() ) + 1) / np.log(10)) + 1)
def get_group(df, cols):
    """
    define an encoding method which can ganrantee the adding value will be unique.
    eg: artist_name_composer will be a combination of (artist_name,composer) and the encoding will reflect the unqiue combination of those two
    """
    group = df[cols[0]].copy()
    for col in cols[1:]:
        print(col)
        group = group * orders[col] + df[col]
        print(group.iloc[0])
        
    return group

def get_group_new(df, cols):
    group = df[cols[0]].copy().apply(str)
    for col in cols[1:]:
        group = group + df[col].apply(str)      
    return group

In [157]:
orders

{'app': 1000,
 'channel': 1000,
 'device': 10000,
 'ip': 1000000,
 'os': 1000,
 'timestamp': 10000000000}

In [173]:
test_group = get_group_new(test,['ip', 'app', 'device', 'channel','os','timestamp'])
test_supplement_group = get_group_new(test_supplement,['ip', 'app', 'device', 'channel','os','timestamp'])

In [174]:
test['encoding'] = test_group
test_supplement['encoding'] = test_supplement_group

In [ ]:
ts1 = 0
ts2 = 6202933  
ts3 = 12316147

In [55]:
start1 = 21290878
start2 = 35678696
start3 = 48109937
end1 = 27493808
end2 = 41791909
end3 = 54584258

# Part 1

In [176]:
# part1 = test.merge(test_supplement.iloc[start1-2: end1+2], on=['ip', 'app', 'device', 'channel','os', 'click_time'], how='inner')
part1 = test.merge(test_supplement.iloc[start1-2: end1+2], on=['encoding'], how='inner')


print(len(part1.click_id_x.value_counts()))
print(len(part1.click_id_y.value_counts()))

6202933
6202933


In [177]:
grouping = part1.groupby('encoding')

In [ ]:
x = []
y = []
count = 0
for group in grouping:
    x.extend(sorted(list(set(group[1].click_id_x.values))))
    y.extend(sorted(list(set(group[1].click_id_y.values))))
    count += 1
    if count % 10000 == 0:
        print(count)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

# Part 2

In [90]:
part2 = test.merge(test_supplement.iloc[start2-2: end2+2], on=['encoding'], how='inner')

print(len(part2.click_id_x.value_counts()))
print(len(part2.click_id_y.value_counts()))

6113214
6113214


In [ ]:
grouping = part2.groupby('encoding')
count = 0
for group in grouping:
    x.extend(sorted(list(set(group[1].click_id_x.values))))
    y.extend(sorted(list(set(group[1].click_id_y.values))))
    count += 1
    if count % 10000 == 0:
        print(count)

# Part 3

In [91]:
part3 = test.merge(test_supplement.iloc[start3-2: end3+2], on=['encoding'], how='inner')

print(len(part3.click_id_x.value_counts()))
print(len(part3.click_id_y.value_counts()))

6474322
6474322


In [ ]:
grouping = part3.groupby('encoding')
count = 0
for group in grouping:
    x.extend(sorted(list(set(group[1].click_id_x.values))))
    y.extend(sorted(list(set(group[1].click_id_y.values))))
    count += 1
    if count % 10000 == 0:
        print(count)

# Check NA

In [5]:
print('training length is {}'.format(len(train)))
print('testing length is {}'.format(len(test)))
print('test_supplement length is {}'.format(len(test_supplement)))

n_train = pd.isnull(train).sum()
n_test = pd.isnull(test).sum()
n_test_supplement = pd.isnull(test_supplement).sum()

print(n_train)
print('-------------------')
print(n_test)
print('-------------------')
print(n_test_supplement)

### there is no N/A

training length is 184903890
testing length is 18790469
test_supplement length is 57537505
ip                         0
app                        0
device                     0
os                         0
channel                    0
click_time                 0
attributed_time    184447044
is_attributed              0
dtype: int64
-------------------
click_id      0
ip            0
app           0
device        0
os            0
channel       0
click_time    0
dtype: int64
-------------------
click_id      0
ip            0
app           0
device        0
os            0
channel       0
click_time    0
dtype: int64


# add time features

In [4]:
# get timestamp

import multiprocessing

def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop('workers')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))

def get_timestamp(x):
    return x.timestamp()





workers = 30

for df in [train, test]:
    clicks = pd.to_datetime(df.click_time)
    print('get clicks')
    df['timestamp'] = apply_by_multiprocessing(clicks, get_timestamp, workers=workers)
#     df['timestamp'] = clicks.apply(lambda t: t.timestamp())
    df['timestamp'] = df['timestamp'].astype('uint32')
    print('timestamping is done')
    
    dt = clicks.dt

    df['year'] = dt.year.astype('uint16')
    print('year is done')
    
    df['month'] = dt.month.astype('uint8')
    print('month is done')
        
    df['week'] = dt.week.astype('uint8')
    print('week is done')
    
    df['day'] = dt.day.astype('uint8')
    print('day is done')
    
    df['hour'] = dt.hour.astype('uint8')
    print('hour is done')
    
    df['minute'] = dt.minute.astype('uint8')
    print('minute is done')
    
    df['second'] = dt.second.astype('uint8')
    print('second is done')
    print('================================================================')

get clicks
timestamping is done
year is done
month is done
week is done
day is done
hour is done
minute is done
second is done
get clicks
timestamping is done
year is done
month is done
week is done
day is done
hour is done
minute is done
second is done


In [6]:
target = 'is_attributed'
feature_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'year', 
              'month',
              'week',
              'day',
              'hour',
              'timestamp',
              'minute',
              'second']

final_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'day',
              'hour',
              'timestamp',
              'minute',
              'second']

train_cols =  feature_col.copy()
train_cols_final =  final_col.copy()
train_cols.append(target)
train_cols_final.append(target)

df_train = train[train_cols]
df_test = test[feature_col]
df_train_final = train[train_cols_final]
df_test_final = test[final_col]

print('saving')
df_train.to_csv(path+'train_cleaned.csv', index=False)
df_train_final.to_csv(path+'train_cleaned_final.csv', index=False)
print('training done')
df_test.to_csv(path+'test_cleaned.csv', index=False)
df_test_final.to_csv(path+'test_cleaned_final.csv', index=False)
print('testing done')

saving
training done
testing done


# processing on sample

In [7]:
train_sample = pd.read_csv(path + 'train_sample.csv',dtype=dtypes)

for df in [train_sample]:
    clicks = pd.to_datetime(df.click_time)
    print('get clicks')
    df['timestamp'] = apply_by_multiprocessing(clicks, get_timestamp, workers=workers)
#     df['timestamp'] = clicks.apply(lambda t: t.timestamp())
    df['timestamp'] = df['timestamp'].astype('uint32')
    print('timestamping is done')
    
    dt = clicks.dt

    df['year'] = dt.year.astype('uint16')
    print('year is done')
    
    df['month'] = dt.month.astype('uint8')
    print('month is done')
        
    df['week'] = dt.week.astype('uint8')
    print('week is done')
    
    df['day'] = dt.day.astype('uint8')
    print('day is done')
    
    df['hour'] = dt.hour.astype('uint8')
    print('hour is done')
    
    df['minute'] = dt.minute.astype('uint8')
    print('minute is done')
    
    df['second'] = dt.second.astype('uint8')
    print('second is done')
    print('================================================================')

get clicks
timestamping is done
year is done
month is done
week is done
day is done
hour is done
minute is done
second is done


In [9]:
df_train_sample = train_sample[train_cols]
df_train_sample.to_csv(path+'train_sample_cleaned.csv', index=False)
df_train_sample_final = df_train_sample[train_cols_final].copy()
df_train_sample_final.to_csv(path+'train_sample_cleaned_final.csv', index=False)